*Ce notebook est distribué par Devlog sous licence Creative Commons - Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions. La description complète de la license est disponible à l'adresse web http://creativecommons.org/licenses/by-nc-sa/4.0/.*

# Initiation Python - Objets 6/6 : Compléments

## Attributs cachés

Dans un objet, les attributs ordinaires sont stockés dans un attribut spécial nommé `__dict__`. Chaque instance possède également un attribut nommé `__class__` qui pointe vers sa classe.

Dans une classe, on trouve également des attributs spéciaux :
* `__doc__`: documentation de la classe,
* `__dict__` :  attributs de la classe,
* `__name__` : nom de la classe, 
* `__bases__` : tuple contenant les classes de bases de la classe courante.
* `__mro__` : ordre de parcours des ancêtres de la classe.
* `__module__` : nom du module où la classe est définie (`__main__` en mode interactif).

In [ ]:
class A1 : pass
class A2 : pass

class B(A1, A2):
    def __init__(self, value):
        self.data = value
    def display(self) : 
        print("data:", self.data)
        
b = B("abc")
print(b.__dict__)

In [ ]:
print(b.__dict__['data'])

In [ ]:
print( b.__class__, )

In [ ]:
print(b.__class__.__name__)

In [ ]:
print(b.__class__.__bases__)

In [ ]:
print(b.__class__.__dict__)

## Attributs pseudo-privés

Au sein d'une instruction composée `class`, tous les noms qui sont préfixés par un double souligné `__` (et non à la fin), sont "magiquement" manipulés par l'interpréteur Python qui insère devant le nom un simple souligné `_` et le nom de la classe courante. Ainsi, les attributs recoivent un nom qui devient spécifique à la classe, et qui ne risque plus d'être redéfini par erreur dans les classes dérivées. On parle d'attributs "pseudo-privés", non pas parce que l'accès à ces attributs est véritablement interdit, mais parce que l'altération automatique de leurs noms complique leur accès de l'extérieur. 

In [ ]:
class Demo:
    public_data = "public data"
    __private_data = "private data"
    def public_method(self): print("public method()")
    def __private_method(self): print("private method()")
    
dir(Demo)

In [ ]:
print(Demo.public_data)

In [ ]:
print(Demo.__private_data)

In [ ]:
print(Demo._Demo__private_data)

In [ ]:
d = Demo()
d.public_method()

In [ ]:
d.__private_method()

In [ ]:
d._Demo__private_method()

À la différence d'autres langages, toutes les données d'un objet réside dans l'objet lui-même, quelle que soit la classe et la méthode à l'origine de ces données. Cela peut générer des interactions involontaires, en particulier dans le cas d'un héritage multiple venant de classes issues de développeurs indépendants. Les attributs pseudo-privés évitent ces interactions non voulues. 

In [ ]:
class C1:
    def set1(self): self.__X = 1
    def get1(self): return self.__X
        
class C2:
    def set2(self): self.__X = 2
    def get2(self): return self.__X
        
class C3(C1, C2):
    # Combien de X ?
    pass

obj = C3()
obj.set1()
obj.set2()
print(obj.get1(), obj.get2())

## Propriétés

On peut définir dans une classe des attributs particuliers de type "propriété" (property). Ces propriétés ressemblent à des variables membres, et s'utilisent comme des variables membres, mais en réalité le fait de lire une propriété déclenche l'utilisation du "getter" qui lui est attaché, et le fait d'affecter une nouvelle valeur à cette propriété déclenche l'utilisation du "setter" attaché (si et seulement si il est défini). Typiquement, "getter" et "setter" manipulent une variable interne pseudo-privée.

Ci-dessous, nous définissons pour la classe `Vector` des propriétés `x` et `y`, dont la lecture déclenche un appel à `getx()` ou `gety()`, qui renvoient les valeurs des variables pseudo-privées `__x` ou `__y` des instances de `Vector`. Seule la propriété `x` a été associée à un "setter". `y`, en l'absence de "setter", est en lecture seule : on ne peut pas lui affecter de valeur (par contre on peut le faire sur `__y` via `init()`).

In [ ]:
class Vector:    
    def init(self,u=0,v=0):
        self.__x = u
        self.__y = v
    def getx(self):
        return self.__x
    def setx(self,u):
        self.__x = u    
    x = property(getx, setx) #Ugly
    def gety(self):
        return self.__y
    y = property(gety)

v = Vector()
v.init(3, -4)
print(v.x, v.y)
v.x = 2
print(v.x)

In [ ]:
v.y = 5

Ce qui est remarquable ici, c'est que Python ne force pas une encapsulation précoce et inutile. On peut très bien développer une première version de classe avec des attributs publics, avoir des clients, puis décider ultérieurement d'en faire des propriétés, si le besoin s'en fait sentir.

On a vu aussi, dans notre exemple, qu'il est facile de faire des propriétés "read-only" en ne déclarant que les "getters". Il est cependant plus classique de fournir à la fois la méthode de lecture et d'écriture, notamment pour controler qu'on n'enregistre que des valeurs valides dans une propriété. Par exemple, si nos vecteurs ne peuvent contenir que des valeurs entre -1 et 1 :

In [ ]:
class Vector:
    def init(self, u=0, v=0):
        self.x = u
        self.y = v
    def getx(self):
        return self.__x
    def setx(self,x):
        if (x<-1): self.__x = -1
        elif (x>1): self.__x = 1
        else: self.__x = x
    x = property(getx,setx)
    def gety(self):
        return self.__y
    def sety(self,y):
        if (y<-1): self.__y = -1
        elif (y>1): self.__y = 1
        else: self.__y = y
    y = property(gety,sety)

v = Vector()
v.init(3, -4)
print(v.x, v.y)

## Variables de classe

On peut ajouter des données à une classe (et pas seulement des méthodes), auquel cas elles sont en quelque sorte partagées et visibles pour toutes les instances de la classe.

En effet, quand vous demandez à lire la donnée nommée `x` de l'instance `obj`, ce qui se note `obj.x`, l'interpréteur Python cherche d'abord le nom dans l'instance elle-même, puis à défaut dans la classe, puis dans ses ancêtres, comme pour n'importe quel attribut.

Dans l'exemple ci-dessous, on dote la classe `Vecteur` de variable `x` et `y` qui contiennent des valeurs par défaut pour les instances. On peut créer ces variables directement lors de la création de classe, comme nous le faisons pour `x`, ou l'ajouter ultérieurement, comme nous le faisons pour `y`.

In [ ]:
class Vecteur:
    x = 0

Vecteur.y = 0

v = Vecteur()

print(Vecteur.x, Vecteur.y)
print(v.x, v.y)

Par contre, si vous affectez une nouvelle valeur à `obj.x`, et que ce nom n'existe pas encore dans l'instance, un nouvel attribut est créé à cette occasion, dans l'instance. Formulons le à nouveau : en cas de lecture, l'interpréteur cherche l'attribut dans l'instance concernée, puis dans sa classe et ses ancêtres, mais en cas d'affectation d'une nouvelle valeur, l'attribut est ajouté à l'instance si il n'existe pas déjà.

In [ ]:
v.x, v.y = 10, 20
Vecteur.z = -1
v.t = 0.5
Vecteur.t = -2

print(Vecteur.x, Vecteur.y)
print(v.x, v.y)
print(v.z, v.t)

## Un langage très dynamique

Les instructions `class` et les instructions `def`, sont traitées à l'exécution comme n'importe quelles instructions. Elles retournent des objets, certes un peu spéciaux, et leur assigne un nom dans l'espace de nom courant. Avec ces objets, on peut réaliser des manipulations impossibles dans la pluapart des autres langages, comme par exemple créer une méthode d'abord en tant que fonction, à l'extérieur de la classe, puis la rattacher à posteriori.

In [ ]:
class A:
    def __init__(self,value): self.data = value
    def display(self) : print("data:", self.data)

a = A('bonjour')

def my_display_upper(self):
    print("data:", self.data.upper())

A.display_upper = my_display_upper

a.display_upper()

Une méthode peut s'invoquer à travers le nom de classe, mais dans ce cas il ne faut pas oublier de redonner le nom de l'objet à traiter comme premier argument (self) :

In [ ]:
A.display_upper(a)

Je peux toujours aussi appeler directement la fonction à travers son nom original (`my_display_upper`) :

In [ ]:
my_display_upper(a)

On peut même s'amuser, comme ci-dessous, à décrocher la fonction de la classe (en effacant le nom), et à l'accrocher directement à l'instance. Mais dans ce cas, la définition automatique de `self` n'est pas réalisée (ce mécanisme n'est actif que pour les fonctions attachées dans des classes). On doit alors redonner `a` comme argument à l'appel de fonction.

In [ ]:
del A.display_upper

a.display_upper = my_display_upper

a.display_upper(a)

## Les encoches

La définition dans une classe d'un attribut spécial nommé `__slots__` permet de limiter la liste des attributs autorisés (sans pour autant les créer). Cela permet d'éviter des erreurs de frappe involontaire, de faire certaines optimisations, mais en contre-partie cela peut affecter l'existence de l'attribut spécial `__dict__`, et perturber le fonctionnement d'outils génériques qui s'appuient sur ce dictionnaire interne.

In [ ]:
class DemoSlots:
    __slots__ = ['att1','att2']

ds = DemoSlots()
print(ds.att1)

In [ ]:
ds.att1 = "this is att1"
print(ds.att1)

In [ ]:
ds.att3 = "this is att3"

In [ ]:
class DerivedSlots(DemoSlots): pass
ds = DerivedSlots()
ds.att3 = "this is att3"
print(ds.att3)

## Méthodes statiques et méthodes de classe

Imaginons que l'on veuille compter le nombre d'instances créées par une classe. Pour récupérer le nombre courant d'instances, on peut écrire une méthode qui ne se sert que de l'attribut de classe, et cette méthode devrait pouvoir être appelée via le nom de classe, sans passer par une instance, mais cela ne fonctionne pas à l'aide d'une implémentation "naïve" telle que celle-ci :

In [ ]:
class DemoComptage:
    __nb_objets = 0
    def __init__(self):
        DemoComptage.__nb_objets = DemoComptage.__nb_objets+1
    def nb_objets():
        return __nb_objets
    
a = DemoComptage()    
b = DemoComptage()    
c = DemoComptage()

print(DemoComptage.nb_objets())

En effet, même si `self` n'est pas utilisé dans le corps d'une méthode, l'interpréteur Python exige qu'une méthode soit invoquée à travers une instance. L'attribut `nb_objets` de la classe est évidemment accessible à tous, donc il peut êtr lu directement par les clients, mais si on tient à préserver l'encapsulation, une simple fonction extérieure peut faire l'affaire :

In [ ]:
class DemoComptage:
    nb_objets = 0
    def __init__(self):
        DemoComptage.nb_objets = DemoComptage.nb_objets+1

def nb_objets():
    return DemoComptage.nb_objets
    
a = DemoComptage()    
b = DemoComptage()    
c = DemoComptage()

print(nb_objets())

Cependant, pour satisfaire les programmeurs qui tiennent à localiser la fonction au sein de la classe, depuis Python 2.2, on peut définir des méthodes dites "statiques", qui peuvent s'invoquer sans passer par une instance :

In [ ]:
class DemoComptage:
    _nb_objets = 0
    def __init__(self):
        DemoComptage._nb_objets = DemoComptage._nb_objets+1
    def nb_objets():
        return DemoComptage._nb_objets
    nb_objets = staticmethod(nb_objets)
    
a = DemoComptage()    
b = DemoComptage()    
c = DemoComptage()

print(DemoComptage.nb_objets())

Il existe également des méthodes dites "de classe", qui recoivent en premier argument non pas l'instance courante, mais la classe courante.

In [ ]:
class DemoClassMethod:
    def m(cls,data):
        print(cls, data)
    m = classmethod(m)
    
DemoClassMethod.m("bonjour")

## Décorateurs de fonctions

Les méthodes statiques et les méthodes de classes sont des cas particuliers de "décorateurs" de fonction.

Un décorateur est une fonction qui manipule une fonction. Il peut effectuer une action une fois pour toute, et renvoyer la fonction originale, ou bien renvoyer une nouvelle fonction qui effectuera des manipulations à chaque appel, avant de le répercuter à la fonction originale qui aura été mémorisée en interne.

Par exemple, lors de l'instruction `nb_objets = staticmethod(nb_objets)`, on appelle le décorateur `staticmethod`, qui substitue à la fonction `nb_objets` originale une autre fonction, dont le rôle sera, à chaque appel, de mettre de côté le premier argument (`self`) et de transmettre les autres à la fonction `nb_objets` originale.

On peut maintenant définir plus facilement une décoration, à l'aide du caractère `@` :

In [ ]:
class DemoDeco:
    _nb_objets = 0
    def __init__(self):
        DemoDeco._nb_objets += 1
    @staticmethod
    def nb_objets():
        return DemoDeco._nb_objets
    @classmethod
    def m(cls,data):
        print(cls, data)
    
a = DemoDeco()    
b = DemoDeco()    
c = DemoDeco()

print(DemoDeco.nb_objets())

In [ ]:
DemoDeco.m("bonjour")

On peut empiler autant de décorateurs que souhaités. Le code ci-dessous :

In [ ]:
@A @B @C
def f():
    ...

est l'équivalent de :

In [ ]:
def f():
    ...
f = A(B(C(f)))

On peut bien sûr écrire ses propres décorateurs. On les implémente en général à l'aide d'une classe qui stocke la fonction décorée, et définit l'opérateur d'appel `__call__`. Par exemple, ci-dessous, une classe qui compte les appels à la fonction décorée :

In [ ]:
class compteur:
    def __init__(self,func):
        self.func = func
        self.count = 0
    def __call__(self,*args):
        self.count += 1
        print('call %s to %s' % (self.count,self.func.__name__))
        self.func(*args)
        
@compteur
def bidon(texte):
    print(texte)

bidon("bonjour")

In [ ]:
bidon("bonsoir")

Enfin, notons qu'il existe des décorateurs permettant de simplifier la définition des propriétés d'une classe (attributs dont l'accès en lecture et/ou en écriture est confié à des méthodes). L'exemple de vecteur précédemment vu peut-être réécrit ainsi :

In [ ]:
class Vector:
    def init(self,u=0,v=0):
        self.x = u
        self.y = v
    @property
    def x(self):
        return self.__x
    @x.setter
    def x(self,u):
        if (u<-1): self.__x = -1
        elif (u>1): self.__x = 1
        else: self.__x = u
    @property
    def y(self):
        return self.__y
    @y.setter
    def y(self,v):
        if (v<-1): self.__y = -1
        elif (v>1): self.__y = 1
        else: self.__y = v

v = Vector()
v.init(3, -4)
print(v.x, v.y)

## Hériter d'un type prédéfini

On peut hériter des types prédéfinis, comme de n'importe quelle autre classe. **ATTENTION**, dans l'exemple ci-dessous, comme on hérite d'une classe "non-modifiable" ("immutable"), on ne peut pas redéfinir **`__init__`**, qui intervient après a création de l'instance ; on est obligé de passer par une redéfinition de **`__new__`**.

In [ ]:
class FloatWithUnit(float):
    def __new__(cls, value, unit):
        instance = float.__new__(cls, value)
        instance.unit = unit
        return instance
    def __str__(self):
        return float.__str__(self)+" "+self.unit
    def __mul__(self,other):
        return FloatWithUnit(float.__mul__(self,other),self.unit+"*"+other.unit)
    
largeur = FloatWithUnit(2,"cm")
longueur = FloatWithUnit(5,"cm")
print(largeur*longueur)

## Les classes de style ancien

Toutes les explications de cette formation concerne les classes de nouveau style, apparue avec 2.2, et qu'il faut utiliser à chaque fois que c'est possible. Comment savoir si une classe est d'ancien ou nouveau style ?
* avant 2.2 : toutes les classes sont de stype ancien
* de 2.2 à 2.x : les classes qui ont la classe `object` parmi leurs ancêtres sont de nouveau style.
* à partir de 3 : toutes les classes sont de nouveau style

Un grand nombre de fonctionnalités avancées ne sont disponibles que pour les classes de nouveau style : `super()`, propriétés, encoches, méthodes statiques, décorateurs, héritage d'un type prédéfini...

Par ailleurs, en cas d'héritage en losange, les comportements son subtilement différents. Pour les classes de nouveau style, la recherche d'attribut se fait en profondeur d'abord, puis de gauche à droite, à une exception près : les classes dérivées sont toujours explorées avant leurs classes de base. Cette exception n'avait pas cours pour les classes de style ancien : 

In [ ]:
class A: x = "A"
class B1(A): pass
class B2(A): x = "B2"
class C(B1,B2): pass

print C.x

En cas de nouveau style : 

In [ ]:
class A(object): x = "A"
class B1(A): pass
class B2(A): x = "B2"
class C(B1,B2): pass

print C.x

Pour éviter toute ambiguité, quel que soit le style de classe, on peut dire explicitement quelle est la méthode à utiliser :

In [ ]:
class A: x = "A"
class B1(A): pass
class B2(A): x = "B2"
class C(B1,B2): x = B2.x

print C.x

## Itérateurs

Aujourd'hui, avant de rechercher une méthode `__getitem__`, l'interpréteur Python cherche d'abord une méthode `__iter__`, qui peut implémenter des schémas d'itération plus complexes. La méthode `__iter__` est supposée retourné un objet "itérateur", sur lequel on va ensuite appeler la méthode `__next__`, qui renvoit un nouvel élément à chaque appel, et lève une exception `StopIteration` lorsqu'il n'y a plus d'éléments.

In [ ]:
class ReverseIterator:
    def __init__(self, seq):
        self.seq = seq
        self.index = len(seq.data)
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.seq.data[self.index]

class Reverse(object):
    def __init__(self, data):
        self.data = data
    def __iter__(self):
        return ReverseIterator(self)

for char in Reverse('123'):
    print(char)

Notre classe peut servir elle aussi dans tous les contextes d'itération.

In [ ]:
inverse = Reverse('123')
[c for c in inverse]

In [ ]:
[c for c in inverse]

In [ ]:
[c+d for c in inverse for d in inverse]

Pour éviter d'avoir à écrire deux classes, on peut être tenté de dire qu'objet est son propre itérateur, et de placer la méthode `__next__()` dans la classe d'origine (exemple ci-dessous). Dans la pratique, c'est peu utile, car l'objet ne peut être itéré qu'une seule fois. On peut alors être tenté d'ajouter une méthode `raz()` qui remettrait l'index à 0, mais avec encore une limite : impossible de lancer deux itérations simultanées. Ne rusez pas : faites deux classes.

In [ ]:
class Reverse:
    def __init__(self, data):
        self.__data = data
        self.__index = len(data)
    def __iter__(self):
        return self
    def __next__(self):
        if self.__index == 0:
            raise StopIteration
        self.__index = self.__index - 1
        return self.__data[self.__index]

inverse = Reverse('123')
print([c for c in inverse])
print([c for c in inverse])

In [ ]:
class Reverse:
    def __init__(self, data):
        self.__data = data
        self.__index = len(data)
    def __iter__(self):
        return self
    def __next__(self):
        if self.__index == 0:
            raise StopIteration
        self.__index = self.__index - 1
        return self.__data[self.__index]
    def raz(self):
        self.__index = len(self.__data)

inverse = Reverse('123')
print([c for c in inverse])
inverse.raz()
print([c for c in inverse])
inverse.raz()
print([c+d for c in inverse for d in inverse])
# Pas de inverse.raz()
print([c for c in inverse])


## A propos des auteurs

*Travail initié en 2014 dans le cadre d'une série de formations Python organisées par le réseau Devlog. Auteur principal : David Chamont. Contribution à la mise à jour pour Python 3 : Fabrice Mendes. Relecteurs : Nicolas Can, Sekou Diakite, Loic Gouarin et Christophe Halgand.*

### Mise en forme

In [ ]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()